In [6]:
import torch
import torch.nn as nn
import tensorflow as tf

# Step 1: Load the Keras model from the HDF5 file
keras_model = tf.keras.models.load_model('cifar10.h5')

# Step 2: Convert the Keras model to a TensorFlow model
tf_model = tf.function(lambda x: keras_model(x))
concrete_function = tf_model.get_concrete_function(tf.TensorSpec(keras_model.layers[0].input_shape))
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_function])
tflite_model = converter.convert()

# Step 3: Extract the weights from the TensorFlow model
tf_weights = []
for layer in keras_model.layers:
    tf_weights.extend(layer.get_weights())

# Step 4: Create a PyTorch model with the same architecture
class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        # Define the architecture based on the Keras model

    def forward(self, x):
        # Implement the forward pass
        pass

# Create an instance of the PyTorch model
pytorch_model = CustomModel()

# Load the weights into the PyTorch model
state_dict = {}
for layer_name, weight in zip(pytorch_model.state_dict().keys(), tf_weights):
    state_dict[f"{layer_name}.weight"] = torch.tensor(weight[0])
    state_dict[f"{layer_name}.bias"] = torch.tensor(weight[1]) if len(weight) > 1 else None

# Load the weights into the PyTorch model
pytorch_model.load_state_dict(state_dict)


# Step 6: Save the PyTorch model to a .pth file
torch.save(pytorch_model.state_dict(), 'converted_model.pth')


2023-12-15 12:02:28.728391: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-15 12:02:28.728521: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-15 12:02:28.730481: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-15 12:02:28.758501: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-15 12:02:30.760320: W tensorflow/compiler/tf2

: 